In [2]:
using Pipe
using Iterators
using DataStructures

In [3]:
function loadphrases(filename)
    (@pipe readall(filename) |> split(_,'\n'))[1:end-1];
end

function load_phrases_and_embeddings(base_filename)
    phrases = loadphrases(base_filename * ".phrases.txt")
    ps = readcsv(base_filename * ".csv")
    @assert size(ps,1)==length(phrases)
    phrases,ps
end

load_phrases_and_embeddings (generic function with 1 method)

In [4]:
#phrase_range = 14380:16000  #old testerment14380:40069
#ps = readcsv("results/gutten.csv")[phrase_range,:]
#phrases = loadphrases("results/gutten.phrases.txt") [phrase_range]

In [66]:
phrases,ps = load_phrases_and_embeddings( "results/boy_destroyed_room")

(SubString{ASCIIString}["The boy destroyed the room.","The boy was responsible for the room's destruction.","The room was destroyed by the boy.","The boy demolished the room.","The room was demolished by the boy.","The boy was responsible for the demolition of the room."],
6x100 Array{Float64,2}:
 -0.1722    -0.15303   -0.0057025  …  -0.070821   0.034731  -0.15358
 -0.054095  -0.10551   -0.096038      -0.079319  -0.060794  -0.18206
 -0.063988  -0.058131  -0.21631       -0.13522   -0.1449    -0.1212 
 -0.17661   -0.15615    0.0097583     -0.01829    0.051056  -0.14609
 -0.069492  -0.065127  -0.21964       -0.13942   -0.1152    -0.1296 
 -0.090505  -0.12887   -0.084873   …  -0.051677  -0.075588  -0.18595)

In [67]:
using PyCall
@pyimport sklearn
@pyimport sklearn.mixture as sklm
@pyimport sklearn.cluster as skl_cluster

In [68]:
function show_clusters(model,words, labels)
    clusters = DefaultDict(Int, Vector{String}, ()->String[])
    for ii in 1:length(words)
        word = words[ii]
        cluster =  labels[ii]
        push!(clusters[cluster], word)
    end
    
    for cluster in keys(clusters)
        println("** $cluster **")
        for word in clusters[cluster]
            println(" - ", word)
        end
        println()
    end
    clusters
end


show_clusters (generic function with 1 method)

In [69]:
using Distances
push!(LOAD_PATH, "../word-embeddings2")
import WordEmbeddings.cosine_sim

In [79]:
pps = ps'
pps_cosine_sim = Float64[WordEmbeddings.cosine_sim(pps[:,ii],pps[:,jj]) for ii in 1:size(pps,2), jj in 1:size(pps,2)];


In [76]:
affinity = pycall(pybuiltin("intern"),PyObject, "precomputed")
affprop = skl_cluster.AffinityPropagation(damping=0.5,affinity=affinity)

affprop[:fit_predict](pps_cosine_sim)
centers = affprop[:cluster_centers_indices_] + 1 #julia indexing
for phrase in phrases[centers]
    println(phrase)
end

The boy was responsible for the room's destruction.
The boy demolished the room.
The room was demolished by the boy.


In [77]:
clusters = show_clusters(affprop,phrases, affprop[:labels_])

** 0 **
 - The boy was responsible for the room's destruction.
 - The boy was responsible for the demolition of the room.

** 2 **
 - The room was destroyed by the boy.
 - The room was demolished by the boy.

** 1 **
 - The boy destroyed the room.
 - The boy demolished the room.



DefaultDict{Int64,Array{String,1},Function} with 3 entries:
  0 => String["The boy was responsible for the room's destruction.","The boy wa…
  2 => String["The room was destroyed by the boy.","The room was demolished by …
  1 => String["The boy destroyed the room.","The boy demolished the room."]

In [55]:
#using TSne
#Y = tsne(ps, no_dims = 2, initial_dims = -1, max_iter = 1000, perplexity = 30.0)
#Y = tsne(ps, 10,  -1, 2000, 20.0)

In [56]:
#using Gadfly
#set_default_plot_size(20cm, 20cm)

#plot(x=Y[:,1], y=Y[:,2],
#        color=affprop[:labels_],
#        label=phrases,
#        Geom.point, Geom.label(position=:centered,hide_overlaps=false))

In [36]:
using Pipe
push!(LOAD_PATH, "../word-embeddings2")
using WordEmbeddings
we = @pipe load_embeddings("../word-embeddings2/word_emb_data/embeddings-scaled.EMBEDDING_SIZE=50.txt") |> WE(_...);



In [37]:
using PyCall
@pyimport nltk
function tokenize(sentence::String)
    convert(Array{String,1},nltk.word_tokenize(sentence))
end

tokenize (generic function with 1 method)

In [91]:
cl_phrases = phrases
cl_wess = [eval_word_embeddings(we, tokenize(sent))' for sent in cl_phrases];


In [93]:
phrases

6-element Array{SubString{ASCIIString},1}:
 "The boy destroyed the room."                            
 "The boy was responsible for the room's destruction."    
 "The room was destroyed by the boy."                     
 "The boy demolished the room."                           
 "The room was demolished by the boy."                    
 "The boy was responsible for the demolition of the room."

In [81]:
@pyimport hmmlearn.hmm as hl_hmm
@pyimport warnings
warnings.filterwarnings("ignore")

In [98]:
hmm = hl_hmm.GMMHMM(n_components=12, n_mix=3, covariance_type="full", n_iter=1000*1000*1000)


PyObject GMMHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
    gmms=None,
    init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
    n_components=12, n_iter=1000000000, n_mix=3,
    params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
    random_state=None, startprob=None, startprob_prior=None, thresh=0.01,
    transmat=None, transmat_prior=None)

In [ ]:
hmm[:fit](cl_wess)


In [96]:
hmm[:monitor_][:history] |> collect |> println
hmm[:monitor_][:n_iter]  |> println
for gmm in hmm[:gmms_]
    println(gmm[:converged_])   
end

{4207.067915070775,3960.427050401919}
100000000
false
false
false
false
false
false
false
false
false
false
false
false


In [97]:
function get_dotfile(hmm,we, min_start_prob = 0.00, min_trans_prob = 0.0)
    if pyisinstance(hmm,pytypeof(hl_hmm.GMMHMM())) 
        function gmm_cores(gmm)
            gmm_means = @pipe gmm[:means_]' |> round(_,5) |> unique(_,2) # first remove any ~duplicate vectors
            show_bests(we,gmm_means,1)[1,1:2:end] |> unique #then remove any duplicate words 
        end
        node_labels = map(gmm_cores, hmm[:gmms_])
    elseif pyisinstance(hmm,pytypeof(hl_hmm.GaussianHMM())) 
        n_means = 3
        node_labels = show_bests(we, hmm[:means_]',n_means)[1:n_means,1:2:end]
        node_labels= [node_labels[:,ii] for ii in 1:size(node_labels,2)]
    else 
        error("Unknown type of HMM")
    end
    
    prob_from_to = @pipe hmm[:_get_transmat]() |> round(_,2)
    function node_text(index::Int, min_start_prob=0)
        text = @pipe node_labels[index] |> join(_,'/')
        start_prob = @pipe hmm[:startprob_][index] |> round(_,2)
        text*= start_prob>min_start_prob ?  "\nStart Prob: $start_prob":""
        text
    end

    graph_text_buff = IOBuffer()
    println(graph_text_buff,"digraph graphname {")

    n_nodes = size(prob_from_to,1)
    for node_index in 1:n_nodes
        lbl = node_text(node_index, min_start_prob)
        println(graph_text_buff,"n$node_index [label= \"$lbl\"];")
    end

    for from_node in 1:n_nodes
        for to_node in 1:n_nodes
            trans_prob = prob_from_to[from_node,to_node]
            if trans_prob>min_trans_prob
                print(graph_text_buff,"n$from_node->n$to_node [")
                print(graph_text_buff,"label= \"$trans_prob\", ")
                print(graph_text_buff,"penwidth = \"$(10*trans_prob)\", ")
                #print(graph_text_buff,"weight = \"$(int(100*trans_prob))\", ")
                print(graph_text_buff, "color = \"$(rand(1)[1]),1.0,0.5\"")
                println(graph_text_buff,"];")
            end
        end
    end
    println(graph_text_buff,"}")
    graph_text_buff |> takebuf_string 
end
get_dotfile(hmm,we) |> print

digraph graphname {
n1 [label= "established/the/.
Start Prob: 0.08"];
n2 [label= "./the/established
Start Prob: 0.08"];
n3 [label= "established/the/.
Start Prob: 0.08"];
n4 [label= "the/established/.
Start Prob: 0.08"];
n5 [label= "./the/established
Start Prob: 0.08"];
n6 [label= "established/the/.
Start Prob: 0.08"];
n7 [label= "the/established/.
Start Prob: 0.08"];
n8 [label= "the/established/.
Start Prob: 0.08"];
n9 [label= "./the/established
Start Prob: 0.08"];
n10 [label= "established/the/.
Start Prob: 0.08"];
n11 [label= "established/the/.
Start Prob: 0.08"];
n12 [label= "established/the/.
Start Prob: 0.08"];
n1->n1 [label= "0.08", penwidth = "0.8", color = "0.1384192049811379,1.0,0.5"];
n1->n2 [label= "0.08", penwidth = "0.8", color = "0.06575101903785008,1.0,0.5"];
n1->n3 [label= "0.08", penwidth = "0.8", color = "0.2441311428672035,1.0,0.5"];
n1->n4 [label= "0.08", penwidth = "0.8", color = "0.9915362573676627,1.0,0.5"];
n1->n5 [label= "0.08", penwidth = "0.8", color = "0.1881

In [65]:
wes, states = hmm[:sample](12)
bests = show_bests(we, wes',1);
@pipe bests[1,1:2:end][:] |> join(_, ' ')

". one still this put this where in it call of writing"